<h1 style="text-align: center;">Scraping Used Cars Data from 'automobile.tn' w/Selenium</h1>

In [ ]:
#pip install bs4

In [ ]:
#pip install selenium

In [ ]:
#pip install --upgrade selenium

In [ ]:
#pip install chromedriver-py

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from chromedriver_py import binary_path
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

import pandas as pd
import re

In [2]:
## ## Main scraping function
####
"""This function calls the necessary functions to scrape the data"""

def scrape_main_page(url, driver):
    driver.get(url)

    all_car_data = []

    while True:
        # Scrape data from the current page
        current_page_data = scrape_current_page(driver)
        all_car_data.extend(current_page_data)

        # Check if there's a next page, exit the loop if not
        if not go_to_next_page(driver):
            break

    return all_car_data

####
## Current page scraping function
####

def scrape_current_page(driver):
    car_data = []

    car_articles = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'occasion-item'))
    )

    for index, car_article in enumerate(car_articles):
        try:
            name_element = car_article.find_element(By.TAG_NAME, 'h2')
            name = name_element.text.strip() if name_element else 'N/A'
            price_element = car_article.find_element(By.CLASS_NAME, 'price')
            price = price_element.text.strip() if price_element else 'N/A'
            year_element = car_article.find_element(By.CLASS_NAME, 'year').find_element(By.CLASS_NAME, 'value')
            year = year_element.text.strip() if year_element else 'N/A'
            kms_element = car_article.find_element(By.CLASS_NAME, 'road').find_element(By.CLASS_NAME, 'value')
            kms = kms_element.text.strip() if kms_element else 'N/A'
            fuel_element = car_article.find_element(By.CLASS_NAME, 'fuel').find_element(By.CLASS_NAME, 'value')
            fuel = fuel_element.text.strip() if fuel_element else 'N/A'
            trns_element = car_article.find_element(By.CLASS_NAME, 'boite').find_element(By.CLASS_NAME, 'value')
            trns = trns_element.text.strip() if trns_element else 'N/A'

            try:
                car_details_link = car_article.find_element(By.CLASS_NAME, 'details-container').get_attribute('href')
                print(car_details_link)

                driver.execute_script("window.open(arguments[0], '_blank');",car_details_link)
                driver.switch_to.window(driver.window_handles[-1])

                car_details = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'technical-details'))
                )

                car_details_html = car_details.get_attribute("outerHTML")
                dl_soup = BeautifulSoup(car_details_html, "lxml")
                
                drivetrain_keywords = ['Traction', 'Propulsion', 'Intégrale']
                trans_pattern = re.compile("|".join(drivetrain_keywords))  # Case-insensitive pattern
                trans = dl_soup.find('td',text=trans_pattern)
                if trans:
                    drvtrn = trans.get_text(strip=True)
                
                cv = dl_soup.find("span",class_="attribute-suffix").parent.text.strip().replace(' CV','')

            except Exception as e:
                print(f"Error getting car details: {e}")
                drvtrn = 'N/A'
                cv = 'N/A'

            finally:
                # Close the current tab (car details)
                driver.close()
                # Switch back to the main tab
                driver.switch_to.window(driver.window_handles[0])

            # Re-fetch the list of car articles after going back to avoid stale elements
            car_articles = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'occasion-item'))
            )

        except StaleElementReferenceException as e:
            print(f"Stale element reference: {e}")
            continue  # Skip this iteration and continue with the next one

        # Append scraped data to list
        car_data.append({'Name': name, 'Price': price, 'Year': year,
                         'Kilometers': kms, 'Fuel': fuel,
                         'Transmission': trns, 'Drivetrain': drvtrn, 'Taxrate': cv})

    return car_data

####
## Move to next page function
####

def go_to_next_page(driver):
    try:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '/html/body/div[1]/div/div[2]/div[6]/div/div[7]/nav/ul/li[7]'))
        )

        # Check if the button is disabled
        if 'disabled' in next_page_button.get_attribute('class'):
            return False

        next_page_link = next_page_button.find_element(By.CLASS_NAME, 'page-link').get_attribute('href') # Get next page link
        driver.get(next_page_link) # Navigate to the next page
        return True
    
    except Exception as e:
        print(f"Error navigating to next page: {e}")
        return False

####    
## Main
####

main_page_url = 'https://www.automobile.tn/fr/occasion' # Main page url

chrome_options = Options()
svc = webdriver.ChromeService(executable_path=binary_path)

driver = webdriver.Chrome(service=svc, options=chrome_options) #Initialize webdriver
driver.maximize_window() #Enable full-size window

try:
    car_data = scrape_main_page(main_page_url, driver) #Call main scraping function
    df = pd.DataFrame(car_data) # Create a dataframe
    print(df) # Print the dataframe
    
except Exception as e:
    print(f"Error: {e}")
finally:
    driver.quit() #Quit the webdriver

https://www.automobile.tn/fr/occasion/mercedes-benz/classe-a/97720
https://www.automobile.tn/fr/occasion/bmw/serie-5/97442
https://www.automobile.tn/fr/occasion/volkswagen/polo-sedan/96818
https://www.automobile.tn/fr/occasion/haval/h6/97517
https://www.automobile.tn/fr/occasion/mercedes-benz/gla/96366
https://www.automobile.tn/fr/occasion/kia/rio-5p/96363
https://www.automobile.tn/fr/occasion/peugeot/301/96018
https://www.automobile.tn/fr/occasion/mercedes-benz/classe-c/95972
https://www.automobile.tn/fr/occasion/volkswagen/t-roc/95930
https://www.automobile.tn/fr/occasion/nissan/qashqai/95008
https://www.automobile.tn/fr/occasion/renault/clio/94295
https://www.automobile.tn/fr/occasion/volkswagen/golf-7/97076
https://www.automobile.tn/fr/occasion/bmw/serie-2-gran-coupe/97693
https://www.automobile.tn/fr/occasion/renault/megane-coupe/97470
https://www.automobile.tn/fr/occasion/mg/gs/97469
https://www.automobile.tn/fr/occasion/audi/a4/97466
https://www.automobile.tn/fr/occasion/cupra/f

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Brand & Model  1636 non-null   object
 1   Price          1636 non-null   object
 2   Year           1636 non-null   object
 3   Kilometers     1636 non-null   object
 4   Fuel Type      1636 non-null   object
 5   Gearbox        1636 non-null   object
 6   Drivetrain     1636 non-null   object
 7   Taxrate        1636 non-null   object
dtypes: object(8)
memory usage: 102.4+ KB


In [22]:
df["Price"] = df["Price"].str.strip().str.replace(" DT", '').str.replace(" ", "").astype(float)
df["Year"] = df["Year"].astype(int)
df["Kilometers"] = df["Kilometers"].str.replace(" ", "").astype(float)
df["Gearbox"] = df["Gearbox"].str.replace("Automatique","Automatic").str.replace("Manuelle","Manual")
df["Fuel Type"] = df["Fuel Type"].str.replace('Essence','Petrol').str.replace('Hybride rechargeable','Plug-in Hybrid')
df['Fuel Type'] = df['Fuel Type'].str.replace('Hybride (diesel)','Diesel Hybrid').str.replace('Hybride (essence)','Petrol Hybrid')
df['Drivetrain'] = df['Drivetrain'].str.replace('Traction','FWD').str.replace('Propulsion','RWD').str.replace('Intégrale','AWD')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Brand & Model  1636 non-null   object 
 1   Price          1636 non-null   float64
 2   Year           1636 non-null   int32  
 3   Kilometers     1636 non-null   float64
 4   Fuel Type      1636 non-null   object 
 5   Gearbox        1636 non-null   object 
 6   Drivetrain     1636 non-null   object 
 7   Taxrate        1636 non-null   object 
dtypes: float64(2), int32(1), object(5)
memory usage: 96.0+ KB


In [20]:
df['Fuel Type'].unique()

array(['Plug-in Hybrid', 'Petrol', 'Diesel', 'Hybride (diesel)',
       'Hybride (essence)'], dtype=object)

In [21]:
df['Gearbox'].unique()

array(['Automatic', 'Manual'], dtype=object)

In [23]:
df['Drivetrain'].unique()

array(['FWD', 'RWD', 'AWD'], dtype=object)

In [24]:
df.to_csv("Used Cars Dataset.csv",index=False,encoding = "utf-8-sig")